In this notebook I work on the question  whether the author of a tweet (very short text) can be successfully identified.
I try to choose the best classification method its parameters set and features

In [ ]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

I choose binary classification problem for the start and use Kim Kardashian and Hillary Clinton tweets

In [ ]:
#KimKardashianTweets data
df_kk=pd.read_csv('../input/KimKardashianTweets.csv')
len(df_kk)

In [ ]:
#HillaryClintonTweets data
df_hc=pd.read_csv('../input/HillaryClintonTweets.csv')
len(df_hc)

First let's choose the same random number of tweets from both authors

In [ ]:
import random
#2000 random sample rows for KK
rows = random.sample(list(df_kk.index), 2000)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(list(df_hc.index), 2000)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

For sparsity reasons I pre-process data before analysis:

 1.  removing re-tweets
 2. removing short messages (less then 4 words)
 3. replacing @ with REF
 4. replacing any url with URL
 5. replacing any date with DATE
 6. replacing any time with TIME
 7. replace digits with NUM

In [ ]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

To avoid overfitting let's hold out a part of the available data as a test set twt_test (X), author_test (Y).

In [ ]:
from sklearn.cross_validation import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df['text'], df['author'], test_size=0.4, random_state=42)

Train set length:

In [ ]:
len(twt_train)

Test set length:

In [ ]:
len(twt_test)

The place to start is to get better results from known classification method that perform well on the problem.
I test several algorithms using its default parameters as well as CountVectorizer() and TfidfTransformer() with default parameters
I also apply ten-fold cross validation on the training set to select the best method

In [ ]:
ScoreSummaryByModel = list()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score

def ModelEvaluation (model,comment):
    pipeline = Pipeline([('vect', CountVectorizer())
                  , ('tfidf', TfidfTransformer())
                  , ('model', model)])
    scores = cross_val_score(pipeline, df['text'], df['author'], cv=10)	
    mean = scores.mean()	
    std = scores.std()	
    #The mean score and the 95% confidence interval of the score estimate (accuracy)
    ScoreSummaryByModel.append([comment,mean, std, "%0.3f (+/- %0.3f)" % (mean, std * 2)])
    print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
ModelEvaluation (MultinomialNB(),'Naive Bayes classifier')

Naive Bayes classifier results are not very impressive. It's just a baseline

In [ ]:
from sklearn.naive_bayes import BernoulliNB
ModelEvaluation (BernoulliNB(binarize=0.0),'Bernoulli Naive Bayes')

Bernoulli Naive Bayes reasults looks better

In [ ]:
from sklearn.svm import LinearSVC
ModelEvaluation (LinearSVC(),'LinearSVC')

LinearSVC looks Ok Maybe parameters tuning of the method improve the results

In [ ]:
from sklearn.svm import SVC
ModelEvaluation (SVC(),'SVC, default rbf kernel')

Ups :( Default rbf kernel definetly does not work

In [ ]:
ModelEvaluation (SVC(kernel='linear'),'SVC, linear kernel')

Well, SVC with linear kernel should be not worse then LinearSVC

In [ ]:
from sklearn.linear_model import SGDClassifier
ModelEvaluation (SGDClassifier(),'SGD')

Linear classifiers (SVM, logistic regression, a.o.) with SGD training should also be good for the problem. Let's try to optimize its parameters

Here is the summary. Bernoulli Naive Bayes with default parameters returned teh best accuracy

In [ ]:
df_ScoreSummaryByModel=DataFrame(ScoreSummaryByModel,columns=['Method','Mean','Std','Accuracy'])
df_ScoreSummaryByModel.sort_values(['Mean'],ascending=False,inplace=True)
df_ScoreSummaryByModel

For better features extractions let's try tokenize the text, remove stopwords (including the placeholders we added at the stage of data pre-processing) and stem the words
I will use the function in farther analysis

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    2. Removes  stopwords
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    stoplist = stopwords.words('english')
    stoplist.append('twitter')
    stoplist.append('pic')
    stoplist.append('com')
    stoplist.append('net')
    stoplist.append('gov')
    stoplist.append('tv')
    stoplist.append('www')
    stoplist.append('twitter')
    stoplist.append('num')
    stoplist.append('date')
    stoplist.append('time')
    stoplist.append('url')
    stoplist.append('ref')

    text_processed = [word.lower() for word in text_processed if word.lower() not in stoplist]
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

I use the grid search approach for parameter tuning. It will methodically build and evaluate a model for each combination of 
algorithm parameters and feature sets.

In [ ]:
ScoreSummaryByModelParams = list()

In [ ]:
from sklearn.grid_search import GridSearchCV
def ModelParamsEvaluation (vectorizer,model,params,comment):
    pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1)
    grid_search.fit(df['text'], df['author'])
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])

In [ ]:
p = {'vect__analyzer':('char', 'char_wb'),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((2, 2), (3, 3)), 
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(CountVectorizer(),BernoulliNB(),p,'Bernoulli Naive Bayes')

It makes sense. 3 chars is almost a word (in actual use, the average is 4.79 letters per word, and 80% are between 2 and 7 letters long according to http://norvig.com/mayzner.html). Other classification methods tests provided the same result - using 3 chars is the best approach. I do not include the results for other methods in the notebook. I tested them on my local computer. Interesting, the score is higher then for word analyzer.

In [ ]:
p = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (3, 3), (5,5),(2,5)), 
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(CountVectorizer(analyzer='word'),BernoulliNB(),p,'Bernoulli Naive Bayes, analyzer=word')

I, actually, expected better results for longer ngrams. Maybe becasue tweets are short messages, unigrams make more sense. Other classification methods tests provided the same result - using unigrams is the best approach. I do not include the results for other methods in the notebook. I tested them on my local computer

In [ ]:
p = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (3, 3), (5,5),(2,5)), 
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(CountVectorizer(analyzer='word',tokenizer=text_process),BernoulliNB(),p,'Bernoulli Naive Bayes, analyzer=word, tokenizer=text_process')

Not a big difference if I use tokenizer. The result even slightly worse. Other classification methods tests provided the same result. I do not include the results for other methods in the notebook. I tested them on my local computer

Now let's try the combination of all vectors with discovered parameters. FeatureUnion concatenates results of multiple transformer objects.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion
word_vector =  CountVectorizer(analyzer='word',stop_words='english',ngram_range=(1, 1),max_df=0.5)
char_vector = CountVectorizer(analyzer='char_wb',ngram_range=(3, 3),max_df=0.75)
vectorizer = FeatureUnion([("chars", char_vector),("words", word_vector)])
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(vectorizer,BernoulliNB(),p,'Bernoulli Naive Bayes, vectorizer')

The result is becomes better on the combination.
Let's try to add tokenized vector

In [ ]:
word_vector =  CountVectorizer(analyzer='word',stop_words='english',ngram_range=(1, 5),max_df=0.5)
char_vector = CountVectorizer(analyzer='char_wb',ngram_range=(3, 3),max_df=0.75)
text_vector = CountVectorizer(analyzer='word',tokenizer=text_process,ngram_range=(3, 3),max_df=0.75)
vectorizer = FeatureUnion([("chars", char_vector),("words", word_vector),("text", text_vector)])
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(vectorizer,BernoulliNB(),p,'Bernoulli Naive Bayes, vectorizer+text_vector')

The result improves a little bit.

Here is the summary. And the winners are:

 1. LinearSVC (C=1, char 3-ngrams)
 2. SVC with linear kernel (C=1, char 3-ngams)
 3. SGDClassifier (alpha: 0.0001, char 3-ngams)
 4. Bernoulli Naive Bayes (alpha = 0.001 based on combination of character 3-ngrams, word unigrams and tokenized unigram)

All 4 of them returns approximately the same accuracy - 0.967 - 0.970
You can find the full summary on my GitHub project page
https://github.com/KaterynaD/TweetsAutorshipAttributionModelsEvaluation/blob/master/Autorship%2Battribution%2Bmodels%2Bevaluation.ipynb
I do not include all tests in Kaggle notebook due to the long running time 
The data may be slightly different of course each run

In [ ]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

Now let's apply the discovered best approach to test data set
I use Bernoulli Naive Bayes and SGDClassifier for comparison

We need score metrics and few functions now

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp
#Visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def ROCCurves (Actual, Predicted):
    '''
    Plot ROC curves for the multiclass problem
    based on http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
    '''
    # Compute ROC curve and ROC area for each class
    n_classes=2
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(Actual, Predicted)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(Actual.ravel(), Predicted.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    ##############################################################################
    # Plot ROC curves for the multiclass problem

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         linewidth=2)

    plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         linewidth=2)

    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")

In [ ]:
def ConfusionMatrix(author_test_b,author_predictions_b):
    cm=confusion_matrix(author_test_b,author_predictions_b)
    plt.matshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
def PredictionEvaluation(author_test_b,author_predictions_b):
    print ('Precision: %0.3f' % (precision_score(author_test_b,author_predictions_b)))
    print ('Accuracy: %0.3f' % (accuracy_score(author_test_b,author_predictions_b)))
    print ('Recall: %0.3f' % (recall_score(author_test_b,author_predictions_b)))
    print ('F1: %0.3f' % (f1_score(author_test_b,author_predictions_b)))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    print ('ROC-AUC: %0.3f' % (roc_auc_score(author_test_b,author_predictions_b)))

Feature Vectors

In [ ]:
word_vector = CountVectorizer(analyzer='word',stop_words='english',ngram_range=(1, 5),max_df=0.5)
char_vector = CountVectorizer(analyzer='char_wb',ngram_range=(3, 3),max_df=0.75)
text_vector = CountVectorizer(analyzer='word',tokenizer=text_process,ngram_range=(3, 3),max_df=0.75)
vectorizer  = FeatureUnion([("chars", char_vector),("words", word_vector),("text", text_vector)])

Bernoulli Naive Bayes pipeline

In [ ]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer()),
    ('clf', BernoulliNB(alpha=0.001)),
    ])

In [ ]:
pipeline.fit(twt_train,author_train)
author_predictions = pipeline.predict(twt_test)

To evaluate the results we need to binarize labels(authors) - 'HillaryClinton' is 0 and 'KimKardashian' is 1

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
author_test_b = lb.fit_transform(author_test.values)
author_predictions_b  = lb.fit_transform(author_predictions)

In [ ]:
PredictionEvaluation(author_test_b,author_predictions_b)

In [ ]:
ROCCurves (author_test_b,author_predictions_b)

In [ ]:
ConfusionMatrix(author_test,author_predictions)

SGDClassifier pipeline on the same test data with slightly higher precision but less accuracy

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char',ngram_range=(3, 3),max_df=1)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(alpha=0.0001, penalty='elasticnet')),
    ])

In [ ]:
pipeline.fit(twt_train,author_train)
author_predictions = pipeline.predict(twt_test)

In [ ]:
author_test_b = lb.fit_transform(author_test.values)
author_predictions_b  = lb.fit_transform(author_predictions)

In [ ]:
PredictionEvaluation(author_test_b,author_predictions_b)

In [ ]:
ROCCurves (author_test_b,author_predictions_b)

In [ ]:
ConfusionMatrix(author_test_b,author_predictions_b)